In [1]:
import pandas as pd
import numpy as np
import re
import json

from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [2]:
data_path = 'C:/nlp/tensorflow-ml-nlp-tf2/5.TEXT_SIM/data_in/'

In [3]:
train_data = pd.read_csv(data_path + 'train.csv', encoding = 'utf-8')
FILTERS = "([~.,!?\"':;)(])"
max_sequence = 31


In [4]:
print(train_data.head())
train_data.shape

   id  qid1  qid2                                          question1  \
0   0     1     2  What is the step by step guide to invest in sh...   
1   1     3     4  What is the story of Kohinoor (Koh-i-Noor) Dia...   
2   2     5     6  How can I increase the speed of my internet co...   
3   3     7     8  Why am I mentally very lonely? How can I solve...   
4   4     9    10  Which one dissolve in water quikly sugar, salt...   

                                           question2  is_duplicate  
0  What is the step by step guide to invest in sh...             0  
1  What would happen if the Indian government sto...             0  
2  How can Internet speed be increased by hacking...             0  
3  Find the remainder when [math]23^{24}[/math] i...             0  
4            Which fish would survive in salt water?             0  


(404290, 6)

In [5]:
train_pos = train_data.loc[train_data['is_duplicate'] == 1]
train_neg = train_data.loc[train_data['is_duplicate'] == 0]

print(train_pos.shape)
print(train_neg.shape)

class_dif = len(train_neg) - len(train_pos)
sample = 1 - (class_dif / len(train_neg))

train_neg = train_neg.sample(frac = sample)

(149263, 6)
(255027, 6)


In [6]:
print("중복 질문 개수: {}".format(len(train_pos)))
print("중복이 아닌 질문 개수: {}".format(len(train_neg)))

중복 질문 개수: 149263
중복이 아닌 질문 개수: 149263


In [7]:
train_data = pd.concat([train_neg, train_pos])

In [8]:
train_data.head()
train_data.shape

(298526, 6)

In [9]:
change_filter = re.compile(FILTERS)

questions1 = [str(s) for s in train_data['question1']]
questions2 = [str(s) for s in train_data['question2']]

f_questions1 = list()
f_questions2 = list()

for q in questions1:
     f_questions1.append(re.sub(change_filter, "", q).lower())
        
for q in questions2:
     f_questions2.append(re.sub(change_filter, "", q).lower())

In [10]:
token = Tokenizer()
token.fit_on_texts(f_questions1 + f_questions2)


In [11]:
q1_sequence = token.texts_to_sequences(f_questions1)
q2_sequence = token.texts_to_sequences(f_questions2)


In [12]:
q1_data = pad_sequences(q1_sequence, maxlen = max_sequence, padding = 'post')
q2_data = pad_sequences(q2_sequence, maxlen = max_sequence, padding = 'post')

In [13]:
word_vocab = {}
word_vocab = token.word_index 
word_vocab["<PAD>"] = 0

labels = np.array(train_data['is_duplicate'], dtype=int)

print('Shape of question1 data: {}'.format(q1_data.shape))
print('Shape of question2 data:{}'.format(q2_data.shape))
print('Shape of label: {}'.format(labels.shape))
print("Words in index: {}".format(len(word_vocab)))

Shape of question1 data: (298526, 31)
Shape of question2 data:(298526, 31)
Shape of label: (298526,)
Words in index: 76487


In [14]:
data_configs = {}
data_configs['vocab'] = word_vocab
data_configs['vocab_size'] = len(word_vocab)

data_configs

{'vocab': {'the': 1,
  'what': 2,
  'is': 3,
  'how': 4,
  'i': 5,
  'to': 6,
  'a': 7,
  'in': 8,
  'do': 9,
  'of': 10,
  'are': 11,
  'and': 12,
  'can': 13,
  'for': 14,
  'you': 15,
  'why': 16,
  'best': 17,
  'my': 18,
  'it': 19,
  'on': 20,
  'does': 21,
  'which': 22,
  'be': 23,
  'if': 24,
  'or': 25,
  'some': 26,
  'get': 27,
  'that': 28,
  'have': 29,
  'should': 30,
  'with': 31,
  'your': 32,
  'an': 33,
  'from': 34,
  'india': 35,
  'will': 36,
  'people': 37,
  'who': 38,
  'when': 39,
  'good': 40,
  'like': 41,
  'would': 42,
  'at': 43,
  'there': 44,
  'as': 45,
  'quora': 46,
  'about': 47,
  'between': 48,
  'one': 49,
  'way': 50,
  'not': 51,
  'most': 52,
  'we': 53,
  'make': 54,
  'did': 55,
  'life': 56,
  'by': 57,
  'where': 58,
  'money': 59,
  'any': 60,
  'was': 61,
  'so': 62,
  'time': 63,
  'me': 64,
  'after': 65,
  'learn': 66,
  'they': 67,
  'this': 68,
  'know': 69,
  'has': 70,
  'difference': 71,
  'whats': 72,
  'new': 73,
  'think': 74,

In [15]:
train_q1_data = 'train_q1.npy'
train_q2_data = 'train_q2.npy'
train_label= 'train_label.npy'
data_config = 'data_configs.json'

np.save(open(data_path + train_q1_data, 'wb'), q1_data)
np.save(open(data_path + train_q2_data, 'wb'), q2_data)
np.save(open(data_path + train_label, 'wb'), train_label)

json.dump(data_configs, open(data_path + data_config, 'w'))

In [16]:
test_data = pd.read_csv(data_path + 'test.csv', encoding='utf-8')
# test_data = test_data.drop(test_data.tail(1217679).index,inplace=True) # drop last n rows

valid_ids = [type(x) == int for x in test_data.test_id] 
test_data = test_data[valid_ids].drop_duplicates()

c:\users\lck11\anaconda3\envs\nlp\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
test_q1 = [str(s) for s in test_data['question1']]
test_q2 = [str(s) for s in test_data['question2']]

f_test_q1 = []
f_test_q2 = []

for q in test_q1:
     f_test_q1.append(re.sub(change_filter, "", q).lower())
        
for q in test_q2:
    f_test_q2.append(re.sub(change_filter, "", q).lower())

In [18]:
test_q1_sequence = token.texts_to_sequences(f_test_q1)
test_q2_sequence = token.texts_to_sequences(f_test_q2)

test_q1_data = pad_sequences(test_q1_sequence, maxlen=max_sequence, padding='post')
test_q2_data = pad_sequences(test_q2_sequence, maxlen=max_sequence, padding='post')

In [19]:
test_id = np.array(test_data['test_id'])

print('Shape of question1 data: {}'.format(test_q1_data.shape))
print('Shape of question2 data:{}'.format(test_q2_data.shape))
print('Shape of ids: {}'.format(test_id.shape))

Shape of question1 data: (2345796, 31)
Shape of question2 data:(2345796, 31)
Shape of ids: (2345796,)


In [20]:
TEST_Q1_DATA = 'test_q1.npy'
TEST_Q2_DATA = 'test_q2.npy'
TEST_ID_DATA = 'test_id.npy'

np.save(open(data_path + TEST_Q1_DATA, 'wb'), test_q1_data)
np.save(open(data_path + TEST_Q2_DATA , 'wb'), test_q2_data)
np.save(open(data_path + TEST_ID_DATA , 'wb'), test_id)